In [1]:
import sys
import os
sys.path.append("../../TychePlot/")
import matplotlib.pyplot as plt
from Plot_processing import calc, export_data, calc_plot_list, add_to_plot
from PDF import PDF
from LifetimePlot import LifetimePlot
from Plot import Plot
import dill
import copy

In [2]:
#obligatory
title=""
name="CsPbBr3"

multithreaded=False

lt_subdir="lt"
output_file_path="lt_plot/"
desiredPlots=[
    {
        "xCol":1,
        #"yCol":2,
        "yCol":3,
        #"custom":{
            #"fitList":[
                #([0,200],[0,200],LifetimePlot.doubleExp,(50,0.5,10**-3, 200, 0.5),(50,0.5),None,{"bounds":([0,0,0,0,0],[100,1,10**-2,5000,1]),"maxfev":100000}),
            #],
        #}
    }
    ]

desiredPlots_lum=[
    {
        "xCol":1,
        "yCol":3,
        #"yCol2":3,
    }
    ]
labelList=[
    
          "DDAF",# CH8000 4000rpm 30s, MoOx 2.5nm, 20nm PVK (3000rpm 5mg/ml), CsPbBr3 native 1mg/ml 500rpm, 55nm B3PYMPM, 0.5nm LiF, Al
        "DDAF 0.1 wt.\,\% KTFSi",# CH8000 4000rpm 30s, MoOx 2.5nm, 20nm PVK (3000rpm 5mg/ml), CsPbBr3:LiTFSi 2:1 500rpm, 55nm B3PYMPM, 0.5nm LiF, Al 
        "DDAF 1 wt.\,\% LiTFSi",# CH8000 4000rpm 30s, MoOx 2.5nm, 20nm PVK (3000rpm 5mg/ml), CsPbBr3:LiTFSi 1:1 500rpm, 55nm B3PYMPM, 0.5nm LiF, Al 
        "DDAF KTFSi LiTFSi mix",#
]


fill="_"

In [3]:
inputParameters={
                        "titleBool":False,
                        "customFontsize":[12,12,8,10,10],
                        "filenamePrefix":output_file_path,
                        "time_domain":"",
                        "useTex":True,
                        "validYCol":[2,3],
                        "normalize_peak":[True,False,True,True],
                        "set_peak_to_zero":[False,True,False,False],
                        "showColAxType":["lin","lin","log","log"],
                        "parse_data_style":"messbox",
                        "showColLabel":["","Time","Voltage","Normalised Luminance"],
                 "showColLabelUnit":["",
                  "Time ({}s)",
                  "Voltage (V)",
                  "Normalised Luminance",             
                 ],
                        "fileFormat":{"separator":"\t", "skiplines":1},

        
}
optionalParameters={
                    "customLims":True, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    None,
                                    None,
                                    None,
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#wavelength
                                    None,#Normalised Intensity
                                    None,
                                  ],
                    "scaled":False
}

In [4]:
groups=[
    (0,),
    (1,),
    (2,),
    (0,1,2,3)
]

group_names=[
    "A_DDAF",
    "B_KTFSi",
    "C_LiTFSi",
    "compare"
]
group_specific_input=[
                        {
                            "desiredPlots":desiredPlots,
                            "inputParameters":inputParameters,
                        }
]*(len(group_names)-1)
group_specific_input+=[
                        {
                            "desiredPlots":desiredPlots_lum,
                            "inputParameters":inputParameters,
                        }
]

In [5]:
fileListPre=sorted(os.listdir(lt_subdir))

In [6]:
keys=[]
lt_fileList=[]
for element in fileListPre:
    key = element.split(fill)[1]
    if key not in keys:
        subFileList=[]
        for element2 in fileListPre:
            if key == element2.split(fill)[1]:
                subFileList.append(os.path.join(lt_subdir,element2))
        keys.append(key)
        lt_fileList.append(subFileList)
        

In [7]:
lt_fileList

[['lt/LXXXIX_A_4.lt'],
 ['lt/LXXXIX_B_4.lt'],
 ['lt/LXXXIX_C_4.lt'],
 ['lt/LXXXIX_D_4.lt']]

In [8]:
plot_prop_list=[]
feature="default"
for group,group_name,specific_input in zip(groups,group_names,group_specific_input):
    fileList=[]
    spectra_loc=[]
    labels_loc=[]
    pixelsizes_loc=[]
    try:
        inputParameters_loc=copy.deepcopy(specific_input["inputParameters"])
    except:
        inputParameters_loc=copy.deepcopy(inputParameters)
    try:
        optionalParameters_loc=copy.deepcopy(specific_input["optionalParameters"])
    except:
        optionalParameters_loc=copy.deepcopy(optionalParameters)
    try:
        desiredPlots_loc=copy.deepcopy(specific_input["desiredPlots"])
    except:
        desiredPlots_loc=copy.deepcopy(desiredPlots)
    for pos in group:
        if isinstance(pos, tuple):
            fileList.append(lt_fileList[pos[0]])
            labels_loc.append(labelList[pos[1]])
            #pixelsizes_loc.append(pixelsizes[pos[3]])
        else:
            fileList.append(lt_fileList[pos])
            labels_loc.append(labelList[pos])
            #pixelsizes_loc.append(pixelsizes[pos])
        #try:
            #if type(spectra_EQE[pos]) == tuple:
                #spectra_loc.append((spectra_subdir+spectra_EQE[pos][0],spectra[pos][1]))
            #else:
                #spectra_loc.append(spectra_subdir+spectra_EQE[pos])
        #except:
            #spectra_loc=""
            #inputParameters_loc["spectralDataFormat"]={"fileEnding":""}
    inputParameters_loc["labels"]=labels_loc
    #inputParameters_loc["spectraFile"]=spectra_loc
    #inputParameters_loc["pixelsize_mm2"]=pixelsizes_loc
    plot_prop_list.append(
        {
             "name":name+fill+group_name,
             "fileList":fileList,
             "desiredPlots":desiredPlots_loc,
             #"present_params":OLED_present,
             "inputParameters":inputParameters_loc,
             "plot_class":LifetimePlot,
             "optionalParameters":optionalParameters_loc,
             "feature":feature,
             "multithreaded":multithreaded
        }
        )
results=calc_plot_list(plot_prop_list, multithreaded=multithreaded)

'LifetimePlot' object has no attribute 'fitterList'


findfont: Font family ['cursive'] not found. Falling back to DejaVu Sans.


'LifetimePlot' object has no attribute 'fitterList'
'LifetimePlot' object has no attribute 'fitterList'
'LifetimePlot' object has no attribute 'fitterList'
'LifetimePlot' object has no attribute 'fitterList'
'LifetimePlot' object has no attribute 'fitterList'
'LifetimePlot' object has no attribute 'fitterList'


<Figure size 432x288 with 0 Axes>

In [9]:
results[3][0][0].dataList

[[<Data.Data at 0x7fc09e4c6be0>],